# 3_EURES_CH_Daten_reinigen

<font color='red'>In diesem Notebook werden die gescrapten Daten (abgespeichert als Data_EURES_CH_17022019.csv) mit Hilfe von Pandas gereinigt und klassifiziert.</font>

### Schritt 1: Daten laden

In [27]:
import pandas as pd

In [28]:
path = "Data_EURES_CH_17022019.csv"

with open(path) as f:
    dateiinhalt = f.read()

In [29]:
df = pd.read_csv(path)

### Schritt 2: Datenstruktur überblicken

In [39]:
df.shape

(4831, 11)

**ERKENNTNIS:** Die Tabelle hat 4831 Einträge, was der Anzahl Jobinserate aus der Schweiz entspricht

In [40]:
df.head(3)

,id,creationDate,lastModificationDate,title,description,numberOfPosts,locationMap,euresFlag,jobCategoriesCodes,employer,score
0,MDgyQ05KQyA5,1547103651000,1550419220000,Conseiller / Conseillère clientèle en assurances,Prospection téléphonique <br>Suivi de la clien...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C3321'],"{'name': 'PHOENIX MANAGEMENT', 'website': None...",NaN
1,MDgyWFJLUSA5,1548745236000,1550419219000,CONSEILLER A LA CLIENTELE (H/F),Analyser la situation globale de vos clients a...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C3321'],"{'name': 'MF CONSEIL SARL', 'website': None, '...",NaN
2,MDgyV0xYUyA5,1548662527000,1550417428000,Téléconseiller / Téléconseillère,Notre club s'oriente aujourd'hui vers un club ...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C5244'],"{'name': 'FRANCE LOISIRS SUISSE', 'website': N...",NaN


In [41]:
df.tail(3)

,id,creationDate,lastModificationDate,title,description,numberOfPosts,locationMap,euresFlag,jobCategoriesCodes,employer,score
4828,MTAwMDAtMTE2NDk0NTI5OC1TIDE,1535144224976,1535144224976,Gesundheits- und Krankenpfleger/in auf der Gyn...,Stellenangebotsbeschreibung: <br>Die Personal ...,2,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C3221'],{'name': 'Personal Spezial Arzt und Pflege Ver...,NaN
4829,MTAwMDAtMTE0ODA0MDA2NS1TIDE,1531547424942,1531547424942,Facharzt (m/w) Innere Medizin (Facharzt/-ärzti...,Stellenangebotsbeschreibung: <br>Die Klinik su...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C2212'],"{'name': 'Connecting People Medical', 'website...",NaN
4830,MTAwMDAtMTE0Nzg5ODUwOC1TIDE,1531547424251,1531547424251,Chefarzt (m/w) Ophtalmologie (Facharzt/-ärztin...,Stellenangebotsbeschreibung: <br>Die Klinik su...,1,{'CH': [None]},False,"['http://data.europa.eu/esco/isco/C2212', 'htt...","{'name': 'Connecting People Medical', 'website...",NaN


**ERKENNTNIS:** Die ersten und die letzten Einträge stimmen überein mit den Jobinseraten auf der ersten und letzten Page auf EURES. Der Scraper scheint alle Jobinserate zu scrapen.

In [42]:
df.dtypes

id                       object
creationDate              int64
lastModificationDate      int64
title                    object
description              object
numberOfPosts             int64
locationMap              object
euresFlag                  bool
jobCategoriesCodes       object
employer                 object
score                   float64
dtype: object

Entscheidend für die Frage, welche Schweizer Jobs im Ausland ausgeschrieben werden sind die Variablen **jobCategoriesCodes**  und **numberOfPosts**.

In [102]:
df['jobCategoriesCodes'].head(100).tail(20)

80    ['http://data.europa.eu/esco/isco/C7112', 'htt...
81    ['http://data.europa.eu/esco/isco/C5131', 'htt...
82            ['http://data.europa.eu/esco/isco/C5151']
83            ['http://data.europa.eu/esco/isco/C3118']
84    ['http://data.europa.eu/esco/isco/C7233', 'htt...
85            ['http://data.europa.eu/esco/isco/C7126']
86    ['http://data.europa.eu/esco/isco/C3118', 'htt...
87    ['http://data.europa.eu/esco/isco/C7421', 'htt...
88            ['http://data.europa.eu/esco/isco/C2654']
89            ['http://data.europa.eu/esco/isco/C2212']
90            ['http://data.europa.eu/esco/isco/C3115']
91            ['http://data.europa.eu/esco/isco/C7123']
92            ['http://data.europa.eu/esco/isco/C7123']
93    ['http://data.europa.eu/esco/isco/C7412', 'htt...
94    ['http://data.europa.eu/esco/isco/C7112', 'htt...
95    ['http://data.europa.eu/esco/isco/C2212', 'htt...
96            ['http://data.europa.eu/esco/isco/C4120']
97            ['http://data.europa.eu/esco/isco/

**ERKENNTNIS:** Wohin führen die Links unter jobCategoriesCodes? Hinter jedem Link (http://data.europa.eu/esco/isco/C7212?) ist auf der Webseite von Eures ein Jobprofil hinterlegt nach dem International Standard Classification of Occupations - ISCO 08 (https://www.bfs.admin.ch/bfs/de/home/statistiken/arbeit-erwerb/nomenclaturen/isco08.html). Damit lassen sich die Daten klassifzieren.

In [43]:
df.describe()

,creationDate,lastModificationDate,numberOfPosts,score
count,4.831000e+03,4.831000e+03,4831.000000,0.0
mean,1.546273e+12,1.548625e+12,1.587663,NaN
std,5.156601e+09,1.661317e+09,1.819530,NaN
min,1.531547e+12,1.531547e+12,1.000000,NaN
25%,1.545318e+12,1.548108e+12,1.000000,NaN
50%,1.548223e+12,1.549062e+12,1.000000,NaN
75%,1.549323e+12,1.549922e+12,1.000000,NaN
max,1.550357e+12,1.550419e+12,40.000000,NaN


In [44]:
df['numberOfPosts'].value_counts().sort_index()

1     4052
2      132
3      170
4      150
5      208
6       14
7        8
8        9
9        3
10      70
11       2
12       4
15       5
20       1
30       2
40       1
Name: numberOfPosts, dtype: int64

In [45]:
df[df['numberOfPosts'] == 40]

,id,creationDate,lastModificationDate,title,description,numberOfPosts,locationMap,euresFlag,jobCategoriesCodes,employer,score
2361,MTAwMDAtMTE2OTExMTU5NS1TIDE,1549063740383,1549063740383,Schweisser/Rohrschlosser/Anlagenbau (Schweißer...,Stellenangebotsbeschreibung: <br>- Isometrie-R...,40,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C7212'],"{'name': 'inova Personal Amriswil GmbH', 'webs...",NaN


In [46]:
df['jobCategoriesCodes'].tail(2470).head(1)

2361    ['http://data.europa.eu/esco/isco/C7212']
Name: jobCategoriesCodes, dtype: object

**ERKENNTNIS:** Die Variable numberOfPosts ist ein Zahlenwert (integer). Sie hat einen Maximalwert von 40. Firmen scheinen zum Teil mehr als eine Stelle unter einem Jobinserat zu inserieren. Eine Firma hat sogar 40 Stellen in einem Jobinserat aufgeschalten. Insgesamt sind unter den 4831 Jobinseraten 7670 offene Stellen subsumiert. Auch wenn 40 Stellen ausgeschrieben sind, können diese alle das gleiche Jobprofil haben.

**AUSNAHME 1:** Weiter Aufschluss geben können die Direktlinks zu den Jobinseraten. Dazu muss nur die id, indiesem Fall MTAwMDAtMTE2OTExMTU5NS1TIDE, in der Url eines beliebigen Jobinserates ausgewechselt werden:  https://ec.europa.eu/eures/eures-searchengine/page/main?lang=en&app=2.1.1p1-build-0#/jv-details/MTAwMDAtMTE2OTExMTU5NS1TIDE

Es handelt sich offenbar um einen Jobvermittler welcher 40 Schlosser sucht.

**AUSNAHME 2:** Gucken wir uns einen weiteren Ausnahmefall an: In einem Jobinserat sind mehr als ein Jobprofil hinterlegt: https://ec.europa.eu/eures/eures-searchengine/page/main?lang=en&app=2.1.1p1-build-0#/jv-details/MTAwMDAtMTE2ODc2OTY1NS1TIDE

Es werden 3 Intesivpfleger gesucht, der Jobinserent hat aber 2 Jobprofile hinterlegt: Health care assistants und Nursing associate professionals.

<font color='red'>**WEITERES VORGEHEN**: Es spielt keine Rolle ob in der Spalte 'jobCategoriesCodes' nur ein oder mehrere Links zu Jobkategorien hinterlegt sind. Wird nur die erste Hierarchiestufe gebraucht, lassen sich die Inserate einfach kategorisieren. Es braucht eine neue Spalte'jobCodes10'wo die einzelnen Inserate der obersten Hierarchiestufe gemäss ISCO 08-Standard zugewiesen werden.</font>

### Schritt 3: Daten reinigen

**VERSUCH 1:** Die Variable 'jobCategoriesCodes' nach dem Komma splitten. (siehe dazu https://stackoverflow.com/questions/14745022/how-to-split-a-column-into-two-columnsDie))

In [6]:
df[['jobCategoriesCodes1','jobCategoriesCodes2']] = pd.DataFrame(df.jobCategoriesCodes.values.tolist(), index= df.index)
print (df)

ValueError: Columns must be same length as key

**Funktioniert nicht!** Warum? Unter 'jobCategoriesCodes' sind Listen mit eintweder einem oder mehreren Elementen abgespeichert. Damit funktioniert obiger Befehl nicht.

**VERSUCH 2:** Die Variable 'jobCategoriesCodes' umwandeln in einen String und dann splitten.

In [7]:
import json

In [8]:
df['jobCategoriesCodes'][93]

"['http://data.europa.eu/esco/isco/C7412', 'http://data.europa.eu/esco/isco/C7411']"

In [11]:
jobCode = json.loads(df['jobCategoriesCodes'][93].replace("'",'"'))[0]
jobCode

'http://data.europa.eu/esco/isco/C7412'

Es funktioniert. Die Variable 'jobCategoriesCodes' ist immer eine Url und gleich aufgebaut:  

http://data.europa.eu/esco/isco/C.....

Man kann also nach '/' splitten, das letzte Element auswählen C7412 und dann die erste Zahl nach dem C.

In [12]:
jobCodeURL = json.loads(df['jobCategoriesCodes'][93].replace("'",'"'))[0]
jobCodeURL

'http://data.europa.eu/esco/isco/C7412'

In [18]:
jobCodeNumber = json.loads(df['jobCategoriesCodes'][93].replace("'",'"'))[0].split("/")[-1][1]
jobCodeNumber

'7'

Jetzt die Funktion auf die ganze Spalte jobCategoriesCodes anwenden und in einer neuen Spalte 'jobCodes10' abspeichern.

In [19]:
df['jobCodes10'] = json.loads(df['jobCategoriesCodes'].replace("'",'"'))[0].split("/")[-1][1]

TypeError: the JSON object must be str, bytes or bytearray, not Series

**Funktioniert nicht!** Warum? json.loads lässt sich nicht auf eine Series anwenden. Es braucht einen Loop.

In [20]:
list_new = []

for var in df['jobCategoriesCodes']:
    list_new.append(json.loads(var.replace("'",'"'))[0].split("/")[-1][1])
list_new


['3',
 '3',
 '5',
 '9',
 '7',
 '7',
 '7',
 '7',
 '8',
 '5',
 '7',
 '9',
 '7',
 '7',
 '8',
 '3',
 '7',
 '7',
 '5',
 '9',
 '5',
 '7',
 '5',
 '7',
 '1',
 '7',
 '7',
 '2',
 '1',
 '2',
 '8',
 '7',
 '7',
 '8',
 '2',
 '7',
 '7',
 '8',
 '4',
 '7',
 '5',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '7',
 '7',
 '7',
 '7',
 '1',
 '5',
 '7',
 '7',
 '1',
 '1',
 '1',
 '4',
 '7',
 '7',
 '2',
 '9',
 '8',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '3',
 '3',
 '7',
 '7',
 '4',
 '7',
 '5',
 '5',
 '3',
 '7',
 '7',
 '3',
 '7',
 '2',
 '2',
 '3',
 '7',
 '7',
 '7',
 '7',
 '2',
 '4',
 '5',
 '3',
 '7',
 '7',
 '7',
 '2',
 '2',
 '2',
 '7',
 '1',
 '3',
 '7',
 '7',
 '8',
 '2',
 '2',
 '7',
 '7',
 '2',
 '2',
 '4',
 '7',
 '9',
 '8',
 '7',
 '7',
 '2',
 '1',
 '1',
 '4',
 '6',
 '5',
 '7',
 '2',
 '3',
 '5',
 '7',
 '2',
 '5',
 '7',
 '9',
 '4',
 '7',
 '1',
 '4',
 '2',
 '2',
 '2',
 '1',
 '2',
 '3',
 '7',
 '5',
 '7',
 '5',
 '5',
 '2',
 '2',
 '3',
 '2',
 '2',
 '5',
 '5',
 '3',
 '7',
 '1',
 '1',
 '2',
 '5',
 '1'

In [22]:
df['jobCodes10'] = list_new

In [26]:
df

,id,creationDate,lastModificationDate,title,description,numberOfPosts,locationMap,euresFlag,jobCategoriesCodes,employer,score,jobCodes10
0,MDgyQ05KQyA5,1547103651000,1550419220000,Conseiller / Conseillère clientèle en assurances,Prospection téléphonique <br>Suivi de la clien...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C3321'],"{'name': 'PHOENIX MANAGEMENT', 'website': None...",NaN,3
1,MDgyWFJLUSA5,1548745236000,1550419219000,CONSEILLER A LA CLIENTELE (H/F),Analyser la situation globale de vos clients a...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C3321'],"{'name': 'MF CONSEIL SARL', 'website': None, '...",NaN,3
2,MDgyV0xYUyA5,1548662527000,1550417428000,Téléconseiller / Téléconseillère,Notre club s'oriente aujourd'hui vers un club ...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C5244'],"{'name': 'FRANCE LOISIRS SUISSE', 'website': N...",NaN,5
3,MDgzUVZCUSA5,1550053870000,1550417428000,Aide de cuisine plongeur H/F,Hôtel familial à la montagne dans le Valais su...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C9412'],"{'name': 'hotel du glacier', 'website': 'http:...",NaN,9
4,MDgxR0pZQiA5,1544693437000,1550413816000,Opérateur régleur-programmeur / Opératrice rég...,VOUS ETES RESPONSABLE DE LA QUALITE ET DE LA P...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C7223'],"{'name': 'Non renseigné', 'website': None, 'se...",NaN,7
5,MDgzSk5RTSA5,1549467071000,1550412028000,Tourneur / Tourneuse sur Commande Numérique -CN-,Pour un organisme de recherche international b...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C7223'],"{'name': 'ALTEAD INDUSTRIES SERVICES', 'websit...",NaN,7
6,MDgyUUJTRCA5,1548082867000,1550412019000,Mécanicien / Mécanicienne outils à découper et...,Pour un organisme de recherche international b...,4,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C7222'],"{'name': 'ALTEAD INDUSTRIES SERVICES', 'websit...",NaN,7
7,MDgzSk1MRyA5,1549465268000,1550412018000,Fraiseur / Fraiseuse sur Commande Numérique -CN-,Pour un organisme de recherche international b...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C7223'],"{'name': 'ALTEAD INDUSTRIES SERVICES', 'websit...",NaN,7
8,MDgzQ0xSUSA5,1549008106000,1550404830000,OPERATEUR MACHINE A INJECTION (H/F),Plasturgie - Injection Gestion de production I...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C8142'],"{'name': 'ncm management', 'website': None, 's...",NaN,8
9,MDgzUk5YUyA5,1550129481000,1550404828000,CHEF DU SERVICE NETTOYAGE,Vous êtes au bénéfice d'une solide expérience ...,1,{'CH': [None]},False,['http://data.europa.eu/esco/isco/C5151'],"{'name': 'AREA TECH SARL', 'website': None, 's...",NaN,5


In [23]:
df.to_csv('Data_jobCodes10.csv', index=False)

<font color='red'>Weiter geht es mit dem **Jupyter Notebook: 4_EURES_CH_Pandas_Plotten**.</font>